
---

# 📘 Snowflake Resource Monitors (Warehouse Monitors)

Think of a **Resource Monitor** as a **budget controller** for your warehouses.

💡 **Analogy:**
Imagine you give your kids a **monthly pocket money limit**.

* They can spend freely until they hit the budget.
* You can choose whether to:

  1. **Warn them** when 80% is gone.
  2. **Stop them from spending more** after 100%.
  3. Or **suspend them immediately** at 100% (no more cash, even mid-purchase).

That’s exactly what Resource Monitors do for warehouses.

---

## 1. Fundamentals

* **What they monitor:**
  Snowflake credits consumed by **warehouses**.
* **Scope:**

  * One monitor → one warehouse.
  * Or → one monitor → multiple warehouses (credits are cumulative).
* **Actions after thresholds:**

  * Just notify (send alert).
  * Suspend the warehouse.
  * Suspend **immediately** (force kill).

---

## 2. Key Concepts

### 🔹 Credit Quota

The total number of credits you allocate to the monitor for a period (e.g., 100 credits per month).

### 🔹 Thresholds

Percentage markers (like 50%, 75%, 100%) where you define actions.

### 🔹 Suspend vs Suspend Immediately

* **Suspend:**

  * Let running queries finish.
  * Don’t allow new queries.
  * Safe stop.

* **Suspend Immediately:**

  * Kill everything immediately (even active queries).
  * Risk: user queries may fail.
  * Use this for strict budgets.

👉 Analogy:

* Suspend = “Finish your exam, but don’t start new ones.”
* Suspend Immediately = “Pens down, stop writing NOW!”

---

## 3. SQL Examples – Creating Resource Monitors

Let’s go step by step with **scenarios**.

---

### Case 1: Basic Resource Monitor for a Single Warehouse

```sql
-- Create a monitor with a quota of 100 credits per month
CREATE RESOURCE MONITOR rm_sales
WITH CREDIT_QUOTA = 100
FREQUENCY = MONTHLY
START_TIMESTAMP = IMMEDIATELY
TRIGGERS ON 75 PERCENT DO NOTIFY
         ON 100 PERCENT DO SUSPEND;
         
-- Attach to a warehouse
ALTER WAREHOUSE sales_wh SET RESOURCE_MONITOR = rm_sales;
```

🔎 Explanation:

* Quota = 100 credits/month.
* At 75% → notify.
* At 100% → suspend warehouse (let queries finish).
* Attached to `sales_wh`.

---

### Case 2: Multiple Warehouses Attached

```sql
CREATE RESOURCE MONITOR rm_analytics
WITH CREDIT_QUOTA = 500
FREQUENCY = MONTHLY
TRIGGERS ON 80 PERCENT DO NOTIFY
         ON 100 PERCENT DO SUSPEND_IMMEDIATE;

ALTER WAREHOUSE bi_wh SET RESOURCE_MONITOR = rm_analytics;
ALTER WAREHOUSE etl_wh SET RESOURCE_MONITOR = rm_analytics;
```

🔎 Explanation:

* Total budget = 500 credits/month.
* Both `bi_wh` and `etl_wh` together consume credits from this pool.
* At 100% → both are **immediately suspended**.
* Useful when you want to control **team-level spend**.

---

### Case 3: Strict Budget Enforcement

```sql
CREATE RESOURCE MONITOR rm_dev
WITH CREDIT_QUOTA = 50
FREQUENCY = DAILY
TRIGGERS ON 100 PERCENT DO SUSPEND_IMMEDIATE;
         
ALTER WAREHOUSE dev_wh SET RESOURCE_MONITOR = rm_dev;
```

🔎 Explanation:

* Dev team gets only **50 credits per day**.
* As soon as they hit 100% → dev warehouse is killed immediately.
* Prevents runaway costs from careless dev queries.

---

### Case 4: Notification Only (Soft Monitoring)

```sql
CREATE RESOURCE MONITOR rm_notify
WITH CREDIT_QUOTA = 200
FREQUENCY = WEEKLY
TRIGGERS ON 50 PERCENT DO NOTIFY
         ON 80 PERCENT DO NOTIFY
         ON 100 PERCENT DO NOTIFY;
         
ALTER WAREHOUSE qa_wh SET RESOURCE_MONITOR = rm_notify;
```

🔎 Explanation:

* Only sends notifications (no suspensions).
* Useful for QA → where you don’t want jobs failing but still want visibility.

---

## 4. Important Parameters Recap

* `CREDIT_QUOTA` → Budget (credits).
* `FREQUENCY` → Reset interval (DAILY, WEEKLY, MONTHLY, YEARLY, NEVER).
* `TRIGGERS` → Thresholds + Actions (NOTIFY, SUSPEND, SUSPEND\_IMMEDIATE).
* `START_TIMESTAMP` → When monitoring begins.
* `END_TIMESTAMP` → Optional end date.

---

## 5. Real-World Scenarios

* **Dev/QA Environments:**
  Always use **daily monitors** with suspend immediate → avoid accidental credit explosions.

* **Prod Environments:**
  Use **monthly monitors** with notify + safe suspend → so you don’t kill critical jobs mid-run.

* **Shared Warehouses (multiple teams):**
  Use one monitor for multiple warehouses → cumulative usage control.

---

# ✅ Summary (Easy to Remember)

* Resource Monitor = **budget controller** for warehouses.
* **Quota** = credits limit.
* **Thresholds** = checkpoints.
* **Suspend** = safe stop.
* **Suspend Immediately** = hard stop.
* Attach to one or many warehouses (cumulative).

---
